# Imports

In [22]:
import numpy
from numpy import arange
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from joblib import dump
from joblib import load

# Descargar Dataset

In [23]:
df = pd.read_csv('/home/user/machinelearning/imida/data/AL41.csv', sep=';')
dataset_validacion = pd.read_csv('/home/user/machinelearning/imida/data/AL41_validation.csv', sep=';', names=df.columns)
dataset_validacion.dropna(inplace=True)
dataset_validacion.head()

,CODEST,FECHA,VVMED,VVMAX,RVIENTO,DVMED,TMIN,TMAX,TMED,TMINABS,...,HRMAXABS,RADMED,RADMAX,HSOL,PREC,HORAS0,HORAS7,DEWPT,DPV,ETO_PM_FAO
0,AL41,01/01/2024,"0,32","4,822","27,99","59,83","3,63","17,67","9,75","2,563",...,"83,8","102,32","514,3",6,0,0,8,"2,26","0,53","0,713398532"
1,AL41,02/01/2024,"0,09","4,665","7,55","168,79","0,74","18,11","7,64","-0,196",...,85,"113,44","797,1",8,0,0,13,"0,76","0,47","0,422334652"
2,AL41,03/01/2024,"0,15","3,744","13,15","168,93","1,92","21,51","9,58","0,92",...,"83,4","123,81","529,7",8,0,0,9,"0,79","0,67","0,548790064"
3,AL41,04/01/2024,"0,06","4,292","4,83","32,82","6,47","17,34","12,18","5,732",...,"86,7","51,55","411,5",6,0,0,2,"7,13","0,43","0,589718576"
4,AL41,05/01/2024,"1,97","9,45","170,4","263,44","7,52","16,55","12,27","6,798",...,"86,8","89,73","704,6",7,0,0,0,"2,53","0,67","1,843253317"


# Pasar a floats

In [24]:
dataset_validacion.replace(',', '.', regex=True, inplace=True)
dataset_validacion = dataset_validacion.astype(str)   #Pasamos todo a String
for variable in dataset_validacion.columns:
    for x in dataset_validacion[variable]:
        if(('.' in x) or (x.isdigit())):   #Sería mejor una expresión regular del estilo [0-9]*[.][0-9]+
            #Si encontramos una variable con un punto, significa que es un float, por lo que lo pasamos a float
            dataset_validacion[variable] = dataset_validacion[variable].astype(float)
            break   #Con detectar una nos vale para cambiar toda la columna

# Columnas para fecha

In [25]:
dataset_validacion['FECHA'] = pd.to_datetime(dataset_validacion['FECHA'], format="%d/%m/%Y")

# extract the day, month, and year components
dataset_validacion['AÑO'] = dataset_validacion['FECHA'].dt.year
dataset_validacion['DIA'] = dataset_validacion['FECHA'].dt.day
dataset_validacion['MES'] = dataset_validacion['FECHA'].dt.month

# show the modified data frame
print(dataset_validacion.dtypes)

CODEST                object
FECHA         datetime64[ns]
VVMED                float64
VVMAX                float64
RVIENTO              float64
DVMED                float64
TMIN                 float64
TMAX                 float64
TMED                 float64
TMINABS              float64
TMAXABS              float64
HRMIN                float64
HRMAX                float64
HRMED                float64
HRMINABS             float64
HRMAXABS             float64
RADMED               float64
RADMAX               float64
HSOL                 float64
PREC                 float64
HORAS0               float64
HORAS7               float64
DEWPT                float64
DPV                  float64
ETO_PM_FAO           float64
AÑO                    int32
DIA                    int32
MES                    int32
dtype: object


# Descargar scalers y models